Generate a spectrogram from a .wav file

In [1]:
# import matplotlib.pyplot as plt
# import IPython.display as ipd 
import librosa
# import librosa.display
import numpy as np


def spectrogram_from_file(file_path, max_freq=10000):
    samples, sampling_rate = librosa.load(file_path, sr=None, mono=True, offset=0.0, duration=None)
    X = librosa.stft(samples)
    return librosa.amplitude_to_db(abs(X))
    
#     plt.figure(figsize=(14, 5))
#     librosa.display.specshow(Xdb, sr=sampling_rate, x_axis='time', y_axis='hz')
#     plt.colorbar()

spectrogram_from_file("audio/hello.wav")

array([[-20.564884  , -29.787586  ,  -7.3620214 , ...,   0.725213  ,
         -0.99411374,  -3.0126836 ],
       [-19.39139   , -14.051232  ,  -8.910709  , ...,  -2.9069686 ,
         -9.905552  ,  -3.6721606 ],
       [-18.209446  , -12.80246   , -25.530844  , ..., -12.095527  ,
         -5.5859337 ,  -4.383841  ],
       ...,
       [-47.91812   , -44.987164  , -48.860184  , ..., -49.969803  ,
        -49.969803  , -47.85353   ],
       [-49.969803  , -49.969803  , -47.564568  , ..., -49.969803  ,
        -49.969803  , -46.341     ],
       [-49.969803  , -49.969803  , -49.870384  , ..., -49.396095  ,
        -44.577984  , -44.82728   ]], dtype=float32)

In [2]:
X = []
y = []

hello_value = 0
goodbye_value = 1

for i in range(1, 21):
    X.append(spectrogram_from_file(f"audio/hello/{i}.wav"))
    y.append(hello_value)

for i in range(1, 21):
    X.append(spectrogram_from_file(f"audio/goodbye/{i}.wav"))
    y.append(goodbye_value)

In [3]:
from sklearn.model_selection import train_test_split

maximum_X = max([i.shape[1] for i in X])
# Pad the values of X with 0s upto the maximum time steps and transpose the matrix
X = [np.pad(i, [(0,0), (0, maximum_X - i.shape[1])], constant_values=(0,)).T for i in X]

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

(30, 196, 1025)

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten

model = Sequential()

model.add(LSTM(units = 50, return_sequences = False, input_shape = (196, 1025)))
model.add(Dropout(0.2))

model.add(Dense(units = 1, activation = "sigmoid"))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=["accuracy"])


model.fit(X_train, y_train, epochs = 50, batch_size = 10)


2022-03-03 11:00:20.903291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-03 11:00:20.903432: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-03 11:00:23.034635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-03 11:00:23.034779: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-03 11:00:23.034798: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ollie-ThinkPad-L490): /proc/driver/nvidia/version does not exist
2022-03-03 11:00:23.036101: I tensorflow/core/platform/cp

Epoch 1/50


2022-03-03 11:00:23.294430: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24108000 exceeds 10% of free system memory.


3/3 [==============================] - 2s 102ms/step - loss: 0.2496 - accuracy: 0.4333
Epoch 2/50
3/3 [==============================] - 0s 94ms/step - loss: 0.2466 - accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 0s 133ms/step - loss: 0.2527 - accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 0s 93ms/step - loss: 0.2510 - accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 0s 97ms/step - loss: 0.2485 - accuracy: 0.5333
Epoch 6/50
3/3 [==============================] - 0s 99ms/step - loss: 0.2494 - accuracy: 0.5667
Epoch 7/50
3/3 [==============================] - 0s 106ms/step - loss: 0.2477 - accuracy: 0.5333
Epoch 8/50
3/3 [==============================] - 0s 98ms/step - loss: 0.2474 - accuracy: 0.5333
Epoch 9/50
3/3 [==============================] - 0s 94ms/step - loss: 0.2469 - accuracy: 0.5667
Epoch 10/50
3/3 [==============================] - 0s 98ms/step - loss: 0.2455 - accuracy: 0.5333
Epoch 11/50
3/3 [====================

In [5]:
# regressor.predict(X_train)
# regressor(X_test).shape
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                215200    
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 215,251
Trainable params: 215,251
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.predict(X_test)

array([[0.02591529],
       [0.56701875],
       [0.4258993 ],
       [0.7155206 ],
       [0.69749135],
       [0.54094255],
       [0.5988152 ],
       [0.54193765],
       [0.47189283],
       [0.55193925]], dtype=float32)

In [7]:
y_test

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0])

In [8]:
model.evaluate(X_test, y_test, verbose = 1)

1/1 [==============================] - 0s 341ms/step - loss: 0.2318 - accuracy: 0.5000


[0.23179180920124054, 0.5]

In [9]:
def evaluate_file(model, file_path):
    spectrogram = np.array(spectrogram_from_file(file_path)).T
    print([spectrogram.shape])
    return model.predict(np.reshape(spectrogram, [1, spectrogram.shape[0], spectrogram.shape[1]]))[0][0]

evaluate_file(model, "audio/hello/hello.wav")

/home/ollie/.local/share/virtualenvs/voice-recognition-TGg8XC8-/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'audio/hello/hello.wav'